<a href="https://colab.research.google.com/github/VladislavTokarev02/technical/blob/main/test_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Тестовое задание для кандидата в команду валидации Альфа-Банка

**Привет!** На связи команда валидации Альфа-Банка. Мы отвечаем за то, чтобы модели, которые используются для принятия решений, были надежными, справедливыми и понятными. Твое задание — провести моделирование и валидацию кредитной модели. Удачи, и помни: мы верим в твои силы! 💪  

---

## Цель  
Проверить навыки анализа данных, валидации ML-моделей и умение формулировать выводы для бизнеса.  

---

## Датасет  
**Название:** [Give Me Some Credit](https://www.kaggle.com/c/GiveMeSomeCredit/data)  
**Описание:**  
Прогнозирование дефолта заемщика на основе финансовых и демографических признаков.  

**Признаки:**  
- `RevolvingUtilizationOfUnsecuredLines` (использование кредитных линий),  
- `age`, `MonthlyIncome`, `NumberOfDependents` и др.  

**Целевая переменная:** `SeriousDlqin2yrs` (дефолт: 0/1).  

---

## Задача  
Проведите валидацию модели кредитного скоринга (логистическая регрессия или градиентный бустинг) и подготовьте отчет.  

---

## Шаги  

### 1. Анализ и предобработка данных  
- Обработайте пропуски (например, в `MonthlyIncome`).  
- Исследуйте выбросы (например, возраст < 18 лет).  
- Визуализируйте распределения ключевых признаков.  
- Предложите методы борьбы с дисбалансом классов.  

### 2. Построение и оценка модели  
- Разделите данные на train/validation/test.  
- Обучите модель (логистическая регрессия или CatBoost/XGBoost).  
- Рассчитайте метрики: **AUC-ROC, Precision, Recall, F1-Score**.  
- Проверьте устойчивость модели через кросс-валидацию (5 folds).  

### 3. Интерпретация и этика  
- Выделите **топ-5 признаков**, влияющих на прогноз (SHAP/LIME).  
- Проверьте логичность влияния признаков (например, высокая долговая нагрузка → выше риск дефолта).  
- Оцените fairness модели: сравните метрики (FPR, TPR) для групп (например, **молодые** vs **старше 40 лет**).  

### 4. Отчет  
Подготовьте общие выводы по проведенной валидации, включив:  
- Выводы о качестве модели и её ограничениях.  
- Рекомендации по улучшению (например, сбор дополнительных данных).  
- Пример: *«Как изменится прогноз, если у заемщика появится иждивенец?»*  

---

## Технические требования  
- Язык: **Python** (Jupyter Notebook).  
- Код должен быть читаемым и содержать комментарии.  

---

## Критерии оценки  
1. Глубина анализа данных и обработки выбросов.  
2. Корректность выбранных метрик и их интерпретация.  
3. Качество визуализаций (распределения, важность признаков).  
4. Практичность рекомендаций в отчете.    

---

**Срок выполнения:** 7 дней.  
**Формат сдачи:**  
- Ноутбук в Collab/GitHub-репозиторий.    

---

Это задание покажет, как вы подходите к анализу реальных данных и делаете ML-модели прозрачными для бизнеса. Ждем твою работу! 🚀  

In [1]:
!pip install ydata_profiling -q

In [2]:
import pandas as pd
import random
import numpy as np
from sklearn.model_selection import train_test_split
from ydata_profiling import ProfileReport
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor

random.seed(42)


pd.set_option('display.max_columns', None)

In [3]:
#data_dict = pd.read_excel('Data Dictionary.xls')

Интерпретация столбцов датасета из data_dict

| Имя переменной                      | Описание                                                                                                                                         |
|-----------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------|
| SeriousDlqin2yrs                    | Лицо имело задолженность 90+ дней или более за последние 2 года.                                                                                      |
| RevolvingUtilizationOfUnsecuredLines| Общий баланс по кредитным картам и персональным кредитным линиям (без недвижимости и займов на покупку авто) делённый на сумму кредитных лимитов.     |
| age                                 | Возраст заёмщика в годах.                                                                                                                            |
| NumberOfTime30-59DaysPastDueNotWorse| Количество случаев, когда заёмщик был в просрочке 30–59 дней, но не более, за последние 2 года.                                                      |
| DebtRatio                           | Соотношение ежемесячных платежей по долгам (алименты, расходы на жизнь и т. д.) к ежемесячному валовому доходу.                                       |
| MonthlyIncome                       | Ежемесячный доход.                                                                                                                                   |
| NumberOfOpenCreditLinesAndLoans     | Количество открытых кредитов (автокредит или ипотека) и кредитных линий (например, кредитные карты).                                                 |
| NumberOfTimes90DaysLate             | Количество случаев, когда заёмщик был в просрочке на 90 дней или более.                                                                              |
| NumberRealEstateLoansOrLines        | Количество займов под недвижимость, включая кредитные линии на недвижимость.                                                                         |
| NumberOfTime60-89DaysPastDueNotWorse| Количество случаев, когда заёмщик был в просрочке 60–89 дней, но не более, за последние 2 года.                                                      |
| NumberOfDependents                  | Количество иждивенцев в семье (исключая самого заёмщика, супругу/супруга и т. д.).                                                                    |

In [4]:
train = pd.read_csv('cs-training.csv', index_col = 0).reset_index(drop = True)
train.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [5]:
test = pd.read_csv('cs-test.csv', index_col = 0).reset_index(drop = True)
test.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,NaN,0.885519,43,0,0.177513,5700.0,4,0,0,0,0.0
1,NaN,0.463295,57,0,0.527237,9141.0,15,0,4,0,2.0
2,NaN,0.043275,59,0,0.687648,5083.0,12,0,1,0,2.0
3,NaN,0.280308,38,1,0.925961,3200.0,7,0,2,0,0.0
4,NaN,1.000000,27,0,0.019917,3865.0,4,0,0,0,1.0


In [6]:
train.shape, test.shape

((150000, 11), (101503, 11))

В качестве первого быстрого ознакомления с данными можно воспользоваться дашбородом в ydata_profiling

In [7]:
profile_report = ProfileReport(train, title='EDA Report')

#profile_report
#вывод дашборда на экран

<b> Рассмотрим более детально датасеты

In [8]:
train.describe()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,1.202690e+05,150000.000000,150000.000000,150000.000000,150000.000000,146076.000000
mean,0.066840,6.048438,52.295207,0.421033,353.005076,6.670221e+03,8.452760,0.265973,1.018240,0.240387,0.757222
std,0.249746,249.755371,14.771866,4.192781,2037.818523,1.438467e+04,5.145951,4.169304,1.129771,4.155179,1.115086
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.029867,41.000000,0.000000,0.175074,3.400000e+03,5.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.154181,52.000000,0.000000,0.366508,5.400000e+03,8.000000,0.000000,1.000000,0.000000,0.000000
75%,0.000000,0.559046,63.000000,0.000000,0.868254,8.249000e+03,11.000000,0.000000,2.000000,0.000000,1.000000
max,1.000000,50708.000000,109.000000,98.000000,329664.000000,3.008750e+06,58.000000,98.000000,54.000000,98.000000,20.000000


In [9]:
test.describe()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,0.0,101503.000000,101503.000000,101503.000000,101503.000000,8.140000e+04,101503.000000,101503.000000,101503.000000,101503.000000,98877.000000
mean,NaN,5.310000,52.405436,0.453770,344.475020,6.855036e+03,8.453514,0.296691,1.013074,0.270317,0.769046
std,NaN,196.156039,14.779756,4.538487,1632.595231,3.650860e+04,5.144100,4.515859,1.110253,4.503578,1.136778
min,NaN,0.000000,21.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,0.030131,41.000000,0.000000,0.173423,3.408000e+03,5.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,0.152586,52.000000,0.000000,0.364260,5.400000e+03,8.000000,0.000000,1.000000,0.000000,0.000000
75%,NaN,0.564225,63.000000,0.000000,0.851619,8.200000e+03,11.000000,0.000000,2.000000,0.000000,1.000000
max,NaN,21821.000000,104.000000,98.000000,268326.000000,7.727000e+06,85.000000,98.000000,37.000000,98.000000,43.000000


In [10]:
train.duplicated().sum(), test.duplicated().sum()
# Полных дубликатов нет

(609, 328)

In [11]:
train.duplicated()==True

,0
0,False
1,False
2,False
3,False
4,False
...,...
149995,False
149996,False
149997,False
149998,False


In [12]:
train.isna().mean().round(4).to_frame().sort_values(by=0, ascending = False).style.format('{:.2%}').background_gradient('coolwarm')

,0
MonthlyIncome,19.82%
NumberOfDependents,2.62%
SeriousDlqin2yrs,0.00%
RevolvingUtilizationOfUnsecuredLines,0.00%
age,0.00%
NumberOfTime30-59DaysPastDueNotWorse,0.00%
DebtRatio,0.00%
NumberOfOpenCreditLinesAndLoans,0.00%
NumberOfTimes90DaysLate,0.00%
NumberRealEstateLoansOrLines,0.00%


In [13]:
test.isna().mean().round(4).to_frame().sort_values(by=0, ascending = False).style.format('{:.2%}').background_gradient('coolwarm')

,0
SeriousDlqin2yrs,100.00%
MonthlyIncome,19.81%
NumberOfDependents,2.59%
RevolvingUtilizationOfUnsecuredLines,0.00%
age,0.00%
NumberOfTime30-59DaysPastDueNotWorse,0.00%
DebtRatio,0.00%
NumberOfOpenCreditLinesAndLoans,0.00%
NumberOfTimes90DaysLate,0.00%
NumberRealEstateLoansOrLines,0.00%


Как мы видим - пропуски есть только в 2 столбцах - ежемесячный доход и количество иждивенцев в семье.

- Количество иждивенцев в семье: доля пропусков немного выше 2.5% - это не столь существенно для анализа, таким образом, можно просто удалить эти пропуски без заполнения. Можно было бы также заполнить модой или выбрать более интересную стратегию, но при таком малом количестве пропусков можно перейти в сторону "чистоты" данных.
- Ежемесячный доход: в простейшем случае можно заполнить медианой, хотя существуют вариации заполнения с учётом k-ближайших соседей, линейной регрессии, k-means. В виду того, что пропусков много (~20% от всех данных) воспользуемся продвинутыми методами.

In [14]:
train.dropna(subset=['NumberOfDependents'], inplace=True)
test.dropna(subset=['NumberOfDependents'], inplace=True)

Первая стратегия: заполнение пропусков MonthlyIncome при помощи KNN

In [15]:
target_column = 'SeriousDlqin2yrs'  # Столбец с таргетом


train_knn = train.copy()
test_knn = test.copy()

# Применяем KNNImputer для заполнения пропусков в столбцах (кроме таргета)
imputer = KNNImputer(n_neighbors=5)

# Применяем KNNImputer к тренировочным данным, исключив таргет
train_knn = pd.DataFrame(imputer.fit_transform(train_knn.drop(columns=[target_column])),
                                columns=train_knn.drop(columns=[target_column]).columns)

# Применяем тот же KNNImputer к тестовым данным, исключив таргет
test_knn = pd.DataFrame(imputer.transform(test_knn.drop(columns=[target_column])),
                               columns=test_knn.drop(columns=[target_column]).columns)

# Добавляем таргет обратно в тестовый набор (тестовые данные не должны быть изменены)
test_knn[target_column] = test[target_column]

In [20]:
train_knn.to_csv('train_knn.csv', index = False)
test_knn.to_csv('test_knn.csv', index = False)

Вторая стратегия: заполнение пропусков MonthlyIncome при помощи случайного леса.

In [23]:
# train_nan = train[train['MonthlyIncome'].isnull()]
# train_no_nan = train.dropna(subset=['MonthlyIncome'])

# # Разделяем данные на признаки (X) и целевую переменную (y)
# X_train = train_no_nan.drop(columns=['MonthlyIncome'])
# y_train = train_no_nan['MonthlyIncome']

# # Инициализация и обучение модели RandomForestRegressor
# rf = RandomForestRegressor(n_estimators=500, random_state=42)
# rf.fit(X_train, y_train)

# # Прогнозируем пропущенные значения в столбце 'MonthlyIncome' для тренировочного набора
# X_nan = train_nan.drop(columns=['MonthlyIncome'])
# predicted_income_train = rf.predict(X_nan)

# # Заполняем пропуски предсказанными значениями в тренировочном наборе
# train_rf = train.copy()
# train_rf.loc[train_rf['MonthlyIncome'].isnull(), 'MonthlyIncome'] = predicted_income_train

# # Прогнозируем пропущенные значения в столбце 'MonthlyIncome' для тестового набора
# X_test = test.drop(columns=['MonthlyIncome'])
# predicted_income_test = rf.predict(X_test)

# # Заполняем пропуски предсказанными значениями в тестовом наборе
# test_rf = test.copy()
# test_rf['MonthlyIncome'] = predicted_income_test

Разберём более детально распределение каждого признака:
- RevolvingUtilizationOfUnsecuredLines: превышение кредитного лимита со стороны заёмщика априори считается нежелательным, в зависимости от финансовой организации критическая доля превышения может варьироваться, однако она не может быть в 50708. Вариант - рассмотреть разные перцентили и сделать вывод, с какого значения можно считать выбросом;
- age: возраст заёмщика также может быть выбросом - лица младше 18 лет не могут брать на себя кредитные обязательства, также важно посмотреть и на верхнюю границу.
- NumberOfTime30-59DaysPastDueNotWorse: посмотрим на распределение значений
- DebtRatio: посмотрим на распределение значений, хотя понятно, что отношение не может сильно превышать единицы.
- MonthlyIncome: посмотрим на распределение значений, хотя ясно, что доход заёмщика не может быть равен 0.
- NumberOfOpenCreditLinesAndLoans: посмотрим на распределение, хотя ясно, что количество открытых кредитов не должно быть слишком большим (например, 58).
- NumberOfTimes90DaysLate: посмотрим на распределение признака.
- NumberRealEstateLoansOrLines: посмотрим на распределение признака.
- NumberOfTime60-89DaysPastDueNotWorse: посмотрим на распределение признака.
- NumberOfDependents: посмотрим на распределение признака.